In [1]:
import os, cv2
import pandas as pd
import numpy as np

In [2]:
# 小轿，面包，卡车，电车，行人，sitting行人，骑行人，杂项
class_name = ["Car", "Van", "Truck", "Tram", "Pedestrian", "Person_sitting", "Cyclist", "Misc", "DontCare"]
class_dict = {key: index for index, key in enumerate(class_name)}

In [3]:
image_path = "/home/hero576/code/python/yolov5/data/kitti/training/image2"
label_path = "/home/hero576/code/python/yolov5/data/kitti/training/label2"
label_output = "/home/hero576/code/python/yolov5/data/kitti/training/yolo_label"

In [ ]:
for file in os.listdir(label_path):
    label_file = os.path.join(label_path,file)
    image_file = os.path.join(image_path,file.replace("txt","png"))
    img = cv2.imread(image_file)
    width = img.shape[1]
    height = img.shape[0]
    with open(label_file) as f:
        for data in f.readlines():
            if not data:
                continue
            label_data = data.split(" ")
            label_class = label_data[0]
            if label_class is None:
                continue
            left, top, right, bottom = float(label_data[4]), float(label_data[5]), float(
                label_data[6]), float(label_data[7])
            cv2.rectangle(img, (int(left),int(top)), (int(right),int(bottom)), (0,255,255))
            cv2.putText(img, label_class, (int(left),int(top)), cv2.FONT_HERSHEY_COMPLEX, 0.45, (0,0,255), 1)
    cv2.imshow("res",img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
class Kitti2Dark:
    Columns_name = ['type', 'truncated', 'occluded', 'alpha', 'bbox_lift', 'bbox_top',
                    'bbox_right', 'bbox_bottom', 'height', 'width', 'length', 'pos_x', 'pos_y', 'pos_z', 'rot_y']

    def __init__(self, image_path, label_path, label_output):
        self.image_path = image_path
        self.label_path = label_path
        self.label_output = label_output
        if not os.path.exists(self.label_output):
            os.makedirs(self.label_output)
        self.convert()

    def convert(self):
        for file in os.listdir(self.label_path):
            img_filename = os.path.join(self.image_path, file.replace("txt", "png"))
            label_filename = os.path.join(self.label_path, file)
            img = cv2.imread(img_filename)
            width = img.shape[1]
            height = img.shape[0]

            with open(label_filename) as f:
                with open(os.path.join(self.label_output, file), "w") as f2:
                    for data in f.readlines():
                        if not data:
                            continue
                        label_data = data.split(" ")
                        label_class = class_dict.get(label_data[0])
                        if label_class is None:
                            continue
                        left, top, right, bottom = float(label_data[4]), float(label_data[5]), float(
                            label_data[6]), float(
                            label_data[7])
                        x = (right + left) / (2 * width)
                        y = (top + bottom) / (2 * height)
                        w = (right - left) / width
                        h = (top - bottom) / height
                        f2.write(" ".join([str(i) for i in [label_class, x, y, w, h]]))
                        f2.write("\n")
k = Kitti2Dark(image_path, label_path, label_output)